In [1]:
import geopandas as gp
%matplotlib inline
import folium

In [2]:
shape = gp.read_file('downloads/RBS_OD_ESB_2017_06/RBS_OD_ESB_2017_06.shp')

In [3]:
host = 'http://fbinter.stadt-berlin.de/fb/wfs/geometry/senstadt/re_schulstand/?service=wfs&request=GetCapabilities'

In [4]:
from owslib.wfs import WebFeatureService
wfs = WebFeatureService(url=host, version='2.0.0')

In [5]:
res = wfs.getfeature(typename='fis:re_schulstand',outputFormat='application/json')
data = res.read()

In [6]:
import json

dic = json.loads(data)

In [7]:
dic['features'][0].keys()

[u'geometry', u'type', u'id', u'properties']

In [8]:
print dic['features'][0]['geometry'].keys()
print dic['features'][0]['geometry']['type']
print dic['features'][0]['geometry']['coordinates']

[u'type', u'coordinates']
Point
[388651.73819065216, 5822447.682920413]


In [9]:
dic['features'][0]['properties']

{u'ADRESSE': u'Tegeler Str. 16',
 u'BEZIRK': u'Mitte',
 u'EMAIL': u'<a href="mailto:1.sps-mitte@gmx.de">1.sps-mitte@gmx.de</a>',
 u'FAX': u'4677779720',
 u'INTERNET': u'',
 u'LEITUNG': u'',
 u'ORTSTEIL': u'Wedding',
 u'PLZ': u'13353',
 u'SCHULART': u'Schulpraktisches Seminar',
 u'SCHULNAME': u'1. Schulpraktisches Seminar (S) Mitte',
 u'TELEFON': u'4677779713',
 u'TRAEGER': u'Bezirk',
 u'ZWEIG_01': u'Schulpraktisches Seminar',
 u'ZWEIG_02': u'',
 u'ZWEIG_03': u'',
 u'ZWEIG_04': u'',
 u'spatial_alias': u'1. Schulpraktisches Seminar (S) Mitte',
 u'spatial_name': u'01A01',
 u'spatial_type': u'Point'}

In [10]:
from pyproj import Proj, transform

inProj = Proj(init='epsg:25833')
outProj = Proj(init='epsg:4326')

schools = {}
for school in dic['features']:
    if school['properties']['SCHULART'] == 'Grundschule':
        coords = transform(inProj,outProj, school['geometry']['coordinates'][0], school['geometry']['coordinates'][1])
        coords = [coords[1],coords[0]]
        schools[school['properties']['SCHULNAME']] = {
            'id': school['properties']['spatial_name'],
            'coord': coords
        }

In [11]:
map_osm = folium.Map(location=[52.5, 13.4],
                     tiles='Stamen Terrain')

folium.GeoJson(shape).add_to(map_osm)
for school, values in schools.items():
    folium.Marker(values['coord'], popup=school).add_to(map_osm)

map_osm